**Typhoon LLM**


In [1]:

!pip install faiss-cpu
!pip install numpy
!pip install openai
!pip install PyPDF2
!pip install sentence_transformers



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip install pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 3.5 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from openai import OpenAI
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# ---- Step 1: Load and Chunk PDF ----
def load_pdf_chunks(pdf_path, chunk_size=500):
    reader = PdfReader(pdf_path)
    full_text = ""
    for page in reader.pages:
        full_text += page.extract_text() + "\n"

    chunks = [full_text[i:i+chunk_size] for i in range(0, len(full_text), chunk_size)]
    return chunks

# ---- Step 2: Embed and Index the Chunks ----
def embed_chunks(chunks, embedder):
    embeddings = embedder.encode(chunks)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))
    return index, embeddings

# ---- Step 3: Retrieve Relevant Chunks ----
def retrieve_context(query, chunks, embedder, index, embeddings, top_k=3):
    query_embedding = embedder.encode([query])
    _, indices = index.search(np.array(query_embedding), top_k)
    return "\n---\n".join([chunks[i] for i in indices[0]])

/Users/natchapon/Documents/rag/rag-besidemom/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
 # ---- RAG Setup ----
pdf_path = "FAQ_for_Chatbot.pdf"  # <-- Replace with your actual PDF
chunks = load_pdf_chunks(pdf_path)
embedder = SentenceTransformer("intfloat/multilingual-e5-large-instruct")
index, embeddings = embed_chunks(chunks, embedder)

# ---- User Query ----
user_input = "ลูกกินนมแม่แล้วถ่ายบ่อยผิดปกติหรือไม่? "
retrieved_context = retrieve_context(user_input, chunks, embedder, index, embeddings)

# ---- OpenAI API Call ----
client = OpenAI(
    api_key="sk-titCpkply6rFBcc6326yqTJzL20JeJJ9ACekZEij4nNCpClA",
    base_url="https://api.opentyphoon.ai/v1"
)

system_prompt = f"""
You are an AI assistant specialized in motherhood and childcare
Use the following additional context to help answer the question:
{retrieved_context}
"""

conversation = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_input}
]

response = client.chat.completions.create(
    model="typhoon-v2-70b-instruct",
    messages=conversation,
    stream=True,
    max_tokens=512
)

for chunk in response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)
print()

ไม่ผิดปกติค่ะ เนื่องจากในนมแม่ประกอบด้วยโปรตีนที่ย่อยง่าย และในน้ำนมแม่ส่วนหน้า ประกอบด้วยน้ำตาลแลคโตสและโอลิโกแซคคาไรด์ ซึ่งช่วยส่งเสริมจุลินทรีย์ชนิดดีในลำไส้ รวมถึงฮอร์โมนพรอสตาแกลนดินที่กระตุ้นการเคลื่อนตัวของลำไส้ ดังนั้น ทารกที่กินนมแม่มักถ่ายบ่อยและอุจจาระเหลวได้ ซึ่งหากอุจจาระของทารกไม่มีมูกเลือด ไม่มีกลิ่นเหม็นเน่า ร่วมกับดูดนมได้ดี ไม่มีไข้ ไม่มีซึม หรือร้องกวนผิดปกติ คุณแม่ก็ไม่ต้องกังวลค่ะ


**Typhoon vs Pathumma**

In [36]:
questions = [
    "ลูกกินนมแม่ แล้วถ่ายบ่อย ผิดปกติหรือไม่ ?",
    "ทารกควรนอนวันละกี่ชั่วโมง",
    "ควรอาบน้ำลูกวันละกี่ครั้ง",
    "ลูกสะอึกบ่อย เกิดจากอะไร ต้องทำอย่างไร",
    "ควรจับเรอลูกทุกครั้งหลังให้นมไหม",
    "ควรให้นมบ่อยแค่ไหน",
    "ควรปรับอุณหภูมิห้องอย่างไรให้เหมาะกับลูก",
    "ควรสังเกตอะไรบ้างในเรื่องการหายใจของลูก",
    "ต้องติดตามพัฒนาการของลูกอย่างไร",
    "ลูกต้องได้รับวัคซีนเหมือนทารกปกติหรือไม่",
    "ควรจัดท่านอนลูกอย่างไร",
    "รู้ได้อย่างไรว่าลูกได้รับนมเพียงพอ",
    "ต้องทำอย่างไรหากลูกน้ำหนักตัวขึ้นช้า",
    "ลูกมีอาการแหวะนมบ่อย ผิดปกติหรือไม่",
    "สามารถอุ้มลูกนาน ๆ ได้ไหม",
    "เทสระบบ",
    "ทำไมลูกดูเหมือนหลับบ่อยผิดปกติ",
    "ควรดูแลลูกอย่างไรเมื่อลูกต้องการออกนอกบ้าน",
    "อาการผิดปกติอะไรที่ต้องสังเกตในทารกคลอดก่อนกำหนด",
    "ลักษณะอุจจาระของทารกคลอดก่อนกำหนดควรเป็นอย่างไร",
    "ต้องทำอย่างไรถ้าลูกถ่ายน้อยหรือไม่ถ่ายหลายวัน?",
    "อุจจาระสีซีดขาวในทารกคลอดก่อนกำหนดถือว่าผิดปกติไหม",
    "ลูกมีอาการเขียวที่ปากและรอบปากตอนกินนม เป็นเรื่องปกติไหม?",
    "จะป้องกันอาการเขียวตอนกินนมได้อย่างไร",
    "ทารกคลอดก่อนกำหนดต้องใช้นมอะไรในการเลี้ยง",
    "ทำไมต้องเพิ่มความเข้มข้นของนมในทารกคลอดก่อนกำหนด",
    "ต้องปรับปริมาณแคลอรีในนมสำหรับทารกคลอดก่อนกำหนดอย่างไร",
    "การปรับนมให้พลังงานสูงขึ้นส่งผลต่อระบบย่อยอาหารของลูกหรือไม่",
    "ควรปรึกษาแพทย์เมื่อไหร่เกี่ยวกับการชงนมและแคลอรีของลูก",
    "จะรู้ได้อย่างไรว่าควรเพิ่มนมให้ลูก ?",
    "ถ้าลูกร้องกวนแต่ไม่ยอมกินนม ควรทำอย่างไร",
    "การจับเรอสำคัญอย่างไรกับลูก ?",
    "อุณหภูมิร่างกายปกติของทารก",
    "รู้ได้อย่างไรว่าลูกมีไข้",
    "การดูแลเบื้องต้นเมื่อทารกมีไข้ควรทำอย่างไร"
]

In [17]:
!pip install openpyxl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [37]:
import pandas as pd
import time
import openpyxl

data_to_excel_70b = []
for question in questions:
   
    retrieved_context = retrieve_context(question, chunks, embedder, index, embeddings)
    system_prompt = f"""
    You are an AI assistant specialized in motherhood and childcare
    Use the following additional context to help answer the question:
    {retrieved_context}
    """
    
    
    conversation = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    
    response = client.chat.completions.create(
        model="typhoon-v2-70b-instruct",
        messages=conversation,
        stream=True,
        max_tokens=512
    )
    full_response = ""
    print(f"\nQuestion: {question}")
    for chunk in response:
        
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="", flush=True)
            full_response += chunk.choices[0].delta.content
           
                    
    data_to_excel_70b.append({
                    'Question': question,
                    'Answer': full_response
                })
    print("\n\n---\n")
    print("full_response:", full_response)
        # Save to Excel
    time.sleep(5)  # To avoid hitting rate limits
print(data_to_excel_70b)
df_70b = pd.DataFrame(data_to_excel_70b)
df_70b.to_excel('typhoon_70b_responses.xlsx', index=False)
print(f"\nSaved {len(data_to_excel_70b)} responses to typhoon_70b_responses.xlsx")


Question: ลูกกินนมแม่ แล้วถ่ายบ่อย ผิดปกติหรือไม่ ?
เป็นปกติค่ะ เนื่องจากในนมแม่ประกอบด้วยโปรตีนที่ย่อยง่าย และในน้ำนมแม่ส่วนหน้า ประกอบด้วยน้ำตาลแลคโตสและโอลิโกแซคคาไรด์ ซึ่งช่วยส่งเสริมจุลินทรีย์ชนิดดีในลำไส้ รวมถึงฮอร์โมนพรอสตาแกลนดินที่กระตุ้นการเคลื่อนตัวของลำไส้ ดังนั้น ทารกที่กินนมแม่มักถ่ายบ่อยและอุจจาระเหลวได้ ซึ่งหากอุจจาระของทารกไม่มีมูกเลือด ไม่มีกลิ่นเหม็นเน่า ร่วมกับดูดนมได้ดี ไม่มีไข้ ไม่มีซึม หรือร้องกวนผิดปกติ คุณแม่ก็ไม่ต้องกังวลค่ะ

---

full_response: เป็นปกติค่ะ เนื่องจากในนมแม่ประกอบด้วยโปรตีนที่ย่อยง่าย และในน้ำนมแม่ส่วนหน้า ประกอบด้วยน้ำตาลแลคโตสและโอลิโกแซคคาไรด์ ซึ่งช่วยส่งเสริมจุลินทรีย์ชนิดดีในลำไส้ รวมถึงฮอร์โมนพรอสตาแกลนดินที่กระตุ้นการเคลื่อนตัวของลำไส้ ดังนั้น ทารกที่กินนมแม่มักถ่ายบ่อยและอุจจาระเหลวได้ ซึ่งหากอุจจาระของทารกไม่มีมูกเลือด ไม่มีกลิ่นเหม็นเน่า ร่วมกับดูดนมได้ดี ไม่มีไข้ ไม่มีซึม หรือร้องกวนผิดปกติ คุณแม่ก็ไม่ต้องกังวลค่ะ

Question: ทารกควรนอนวันละกี่ชั่วโมง
ทารกควรนอนวันละประมาณ 16 - 20 ชั่วโมงต่อวัน โดยจะตื่นเพื่อกินนมทุก 2 - 3 ชั่วโมง หา

In [38]:
import pandas as pd
import time
import openpyxl

data_to_excel_12b = []
for question in questions:
   
    retrieved_context = retrieve_context(question, chunks, embedder, index, embeddings)
    system_prompt = f"""
    You are an AI assistant specialized in motherhood and childcare
    Use the following additional context to help answer the question:
    {retrieved_context}
    """
    
    
    conversation = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    
    response = client.chat.completions.create(
        model="typhoon-v2.1-12b-instruct",
        messages=conversation,
        stream=True,
        max_tokens=512
    )
    full_response = ""
    print(f"\nQuestion: {question}")
    for chunk in response:
        
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="", flush=True)
            full_response += chunk.choices[0].delta.content
           
                    
    data_to_excel_12b.append({
                    'Question': question,
                    'Answer': full_response
                })
    print("\n\n---\n")
    print("full_response:", full_response)
        # Save to Excel
    time.sleep(5)  # To avoid hitting rate limits
print(data_to_excel_12b)
df_12b = pd.DataFrame(data_to_excel_12b)
df_12b.to_excel('typhoon_12b_responses.xlsx', index=False)
print(f"\nSaved {len(data_to_excel_12b)} responses to typhoon_12b_responses.xlsx")


Question: ลูกกินนมแม่ แล้วถ่ายบ่อย ผิดปกติหรือไม่ ?
เป็นปกติค่ะ เนื่องจากในนมแม่ประกอบด้วยโปรตีนที่ย่อยง่าย และในน้ำนมแม่ส่วนหน้า ประกอบด้วยน้ำตาลแลคโตส และโอลิโกแซคคาไรด์ ซึ่งช่วยส่งเสริมจุลินทรีย์ชนิดดีในลำไส้ รวมถึงฮอร์โมนพรอสตาแกลนดินที่กระตุ้นการเคลื่อนตัวของลำไส้ ดังนั้น ทารกที่กินนมแม่มักถ่ายบ่อยและอุจจาระเหลวได้

ซึ่งหากอุจจาระของทารกไม่มีมูกเลือด ไม่มีกลิ่นเหม็นเน่า ร่วมกับดูดนมได้ดี ไม่มีไข้ ไม่มีซึม หรือร้องกวนผิดปกติ คุณแม่ก็ไม่ควรรีบพาลูกไปพบแพทย์ค่ะ

---

full_response: เป็นปกติค่ะ เนื่องจากในนมแม่ประกอบด้วยโปรตีนที่ย่อยง่าย และในน้ำนมแม่ส่วนหน้า ประกอบด้วยน้ำตาลแลคโตส และโอลิโกแซคคาไรด์ ซึ่งช่วยส่งเสริมจุลินทรีย์ชนิดดีในลำไส้ รวมถึงฮอร์โมนพรอสตาแกลนดินที่กระตุ้นการเคลื่อนตัวของลำไส้ ดังนั้น ทารกที่กินนมแม่มักถ่ายบ่อยและอุจจาระเหลวได้

ซึ่งหากอุจจาระของทารกไม่มีมูกเลือด ไม่มีกลิ่นเหม็นเน่า ร่วมกับดูดนมได้ดี ไม่มีไข้ ไม่มีซึม หรือร้องกวนผิดปกติ คุณแม่ก็ไม่ควรรีบพาลูกไปพบแพทย์ค่ะ

Question: ทารกควรนอนวันละกี่ชั่วโมง
ทารกมักนอนประมาณ 16 - 20 ชั่วโมงต่อวัน โดยจะตื่นเพื่อกินน

In [14]:
!pip install openpyxl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [34]:
import pandas as pd

# Sample data
data = [{'Question': 'ลูกกินนมแม่ แล้วถ่ายบ่อย ผิดปกติหรือไม่ ?', 'Answer': 'ไม่ผิดปกติค่ะ เนื่องจากในนมแม่ประกอบด้วยโปรตีนที่ย่อยง่าย และในน้ำนมแม่ส่วนหน้า ประกอบด้วยน้ำตาลแลคโตสและโอลิโกแซคคาไรด์ ซึ่งช่วยส่งเสริมจุลินทรีย์ชนิดดีในลำไส้ รวมถึงฮอร์โมนพรอสตาแกลนดินที่กระตุ้นการเคลื่อนตัวของลำไส้ ดังนั้น ทารกที่กินนมแม่มักถ่ายบ่อยและอุจจาระเหลวได้ ซึ่งหากอุจจาระของทารกไม่มีมูกเลือด ไม่มีกลิ่นเหม็นเน่า ร่วมกับดูดนมได้ดี ไม่มีไข้ ไม่มีซึม หรือร้องกวนผิดปกติ คุณแม่ก็ไม่ต้องกังวลค่ะ'}, {'Question': 'ทารกควรนอนวันละกี่ชั่วโมง', 'Answer': 'ทารกควรนอนวันละประมาณ 16 - 20 ชั่วโมงต่อวัน โดยจะตื่นเพื่อกินนมทุก 2 - 3 ชั่วโมง หากลูกนอนหลับเป็นช่วง ๆ ตื่นมาดูดนมได้ดีและน้ำหนักเพิ่มตามเกณฑ์ ถือว่าปกติค่ะ'}, {'Question': 'ควรอาบน้ำลูกวันละกี่ครั้ง', 'Answer': 'ทารกควรอาบน้ำวันละ 1 ครั้งก็เพียงพอ ควรใช้น้ำอุ่นและอาบในห้องที่ไม่มีลมโกรก หลังอาบน้ำควรเช็ดตัวให้แห้งทันทีเพื่อป้องกันการหนาวเย็น'}]

# Create DataFrame
df = pd.DataFrame(data)

# Save to Excel (requires openpyxl)
df.to_excel("people.xlsx", index=False)

print("Excel file 'people.xlsx' saved successfully.")


Excel file 'people.xlsx' saved successfully.
